In [1]:
from mutagen.id3 import ID3, APIC

input_file = "/home/dgmastertemple/Documents/GitHub/create-audio-bible/projects/Ephesians - Tim Conway/export/verses/Ephesians 1:1 - Tim Conway.mp3"
cover_image = "/home/dgmastertemple/Documents/GitHub/create-audio-bible/sources/ill_be_honest.jpeg"

def add_album_art(input_file: str, cover_image: str):
    audio = ID3(input_file)
    with open(cover_image, 'rb') as image_file:
        image_data = image_file.read()
    mime='image/jpeg'
    if cover_image.endswith(".png"):
        mime='image/png'
    album_art = APIC(
        encoding=3,  # UTF-8
        mime=mime,
        type=3,  # Cover (front)
        desc='Cover (Front)',
        data=image_data
    )
    audio['APIC'] = album_art
    audio.save(input_file, v2_version=3)

add_album_art(input_file, cover_image)

In [3]:
from mutagen.mp3 import MP3
from mutagen.id3 import ID3, APIC

def set_album_cover(mp3_path, image_path):
    # Load the MP3 file
    audio = MP3(mp3_path, ID3=ID3)

    # Add ID3 tag if it doesn't exist
    try:
        audio.add_tags()
    except:
        pass

    # Open the image file
    with open(image_path, 'rb') as image_file:
        image_data = image_file.read()

    # Create the APIC frame (album cover)
    album_cover = APIC(
        encoding=3,  # UTF-8
        mime='image/jpeg',  # Or 'image/png' for PNG files
        type=3,  # 3 is for the cover image
        desc='Cover',
        data=image_data
    )

    # Remove existing APIC frames
    audio.tags.delall('APIC')

    # Add the new APIC frame
    audio.tags.add(album_cover)

    # Save the updated MP3 file
    audio.save()

# Usage example
mp3_file_path = 'path/to/your/song.mp3'
image_file_path = 'path/to/your/cover_image.jpg'

set_album_cover(input_file, cover_image)

In [8]:
from mutagen.mp3 import MP3

mp3_file_path = input_file

audio = MP3(mp3_file_path)
album_art = audio.tags.get('APIC', None)

if album_art:
    print(f"Album Art Type: {album_art.type}")
    print(f"Album Art MIME Type: {album_art.mime}")
    print(f"Album Art Description: {album_art.desc}")
    print(f"Album Art Data Length: {len(album_art.data)}")
else:
    print("No album art found in the MP3 file.")

No album art found in the MP3 file.


In [9]:
from mutagen.mp3 import MP3
from mutagen.id3 import ID3, APIC, error
from PIL import Image
import io

def set_album_cover(mp3_path, image_path):
    # Open the image file and ensure it's in the right format (JPEG or PNG)
    with open(image_path, 'rb') as img_file:
        img_data = img_file.read()

    # Determine the MIME type of the image
    img = Image.open(io.BytesIO(img_data))
    mime_type = Image.MIME.get(img.format)

    if not mime_type:
        raise ValueError(f"Unsupported image format: {img.format}")

    # Load the MP3 file
    audio = MP3(mp3_path, ID3=ID3)

    # Add ID3 tag if it doesn't exist
    if audio.tags is None:
        audio.add_tags()

    # Remove existing album art
    audio.tags.delall('APIC')

    # Add the album art
    audio.tags.add(
        APIC(
            encoding=3,  # 3 is for utf-8
            mime=mime_type,  # image/jpeg or image/png
            type=3,  # 3 is for the album cover
            desc=u'Cover',
            data=img_data
        )
    )

    # Save the changes to the MP3 file
    audio.save(v2_version=3)  # Save with ID3v2.3 tag

# Example usage
mp3_path = input_file
image_path = cover_image  # or image.png

set_album_cover(mp3_path, image_path)


In [10]:
import mutagen
from mutagen.id3 import ID3, TXXX

def add_personal_note(mp3_file, note):
    """
    Adds a personal note as metadata to an MP3 file.

    Args:
        mp3_file (str): The path to the MP3 file.
        note (str): The personal note to be added.
    """
    try:
        audio = mutagen.File(mp3_file, easy=True)
        if audio.tags is None:
            audio.add_tags()

        audio.tags.add(TXXX(encoding=3, desc='Personal Note', text=note))
        audio.save()
        print(f"Personal note added to {mp3_file}")
    except Exception as e:
        print(f"Error adding personal note: {e}")

def read_personal_note(mp3_file):
    """
    Reads the personal note metadata from an MP3 file.

    Args:
        mp3_file (str): The path to the MP3 file.

    Returns:
        str: The personal note, or None if no note is found.
    """
    try:
        audio = mutagen.File(mp3_file, easy=True)
        if audio.tags is not None:
            for tag in audio.tags.get('TXXX', []):
                if tag.desc == 'Personal Note':
                    return tag.text[0]
        return None
    except Exception as e:
        print(f"Error reading personal note: {e}")
        return None

add_personal_note(input_file, "this is a test note: 123456")

Error adding personal note: 'EasyID3' object has no attribute 'add'


In [16]:
from mutagen.id3 import ID3, TIT2

def add_personal_note(mp3_file, note):
  """
  Adds a personal note to an MP3 file using ID3v2 tags.

  Args:
    mp3_file (str): Path to the MP3 file.
    note (str): The personal note to add.
  """
  try:
    audio = ID3(mp3_file)
    # Add note using a standard title frame (TIT2) with a custom prefix
    audio.add(TIT2(text=f"Personal Note: {note}"))
    audio.save()
    print(f"Note added to {mp3_file}")
#   except ID3NoTagError:
#     print(f"Error: {mp3_file} has no ID3 tag. Note not added.")
  except Exception as e:
    print(f"Error adding note to {mp3_file}: {e}")

def read_personal_note(mp3_file):
  """
  Reads the personal note from an MP3 file using ID3v2 tags.

  Args:
    mp3_file (str): Path to the MP3 file.

  Returns:
    str: The personal note, or None if not found.
  """
  try:
    audio = ID3(mp3_file)
    for frame in audio.getall("TIT2"):
      print(frame.text)
      if frame.text[0].startswith("Personal Note: "):
        return frame.text[0][len("Personal Note: "):]
    print(f"No personal note found in {mp3_file}")
    return None
#   except ID3NoTagError:
#     print(f"Error: {mp3_file} has no ID3 tag. Note not found.")
  except Exception as e:
    print(f"Error reading note from {mp3_file}: {e}")
    return None

# Example usage
mp3_file = input_file
note = "This song reminds me of summer vacation."

add_personal_note(mp3_file, note)
personal_note = read_personal_note(mp3_file)

if personal_note:
  print(f"Your personal note: {personal_note}")


Note added to /home/dgmastertemple/Documents/GitHub/create-audio-bible/projects/Ephesians - Tim Conway/export/verses/Ephesians 1:1 - Tim Conway.mp3
['Personal Note: This song reminds me of summer vacation.']
Your personal note: This song reminds me of summer vacation.


In [ ]:
from mutagen.id3 import ID3, TIT2

def add_source_data(mp3_file, note):
  """
  Description:
    Adds source data to mp3 so I know how it was generated so I know if I need to regenerate it
  Parameters:
    mp3_file: str - Path to the MP3 file.
    note: str - The note to add.
  """
  try:
    audio = ID3(mp3_file)
    audio.add(TIT2(text=f"source=[{note}]"))
    audio.save()
  except Exception as e:
    print(f"Error adding note to {mp3_file}: {e}")

def read_source_data(mp3_file):
  """
  Description:
    Reads source data from mp3 so I know how it was generated so I know if I need to regenerate it
  Args:
    mp3_file: str - Path to the MP3 file.
  Returns:
    str: Source data
  """
  try:
    audio = ID3(mp3_file)
    for frame in audio.getall("TIT2"):
      if frame.text[0].startswith("source=["):
        return frame.text[0][len("source=["):-1].split(",")
    return None
  except Exception as e:
    print(f"Error reading note from {mp3_file}: {e}")
    return None

# Example usage
mp3_file = input_file
note = "This song reminds me of summer vacation."

add_personal_note(mp3_file, note)
personal_note = read_personal_note(mp3_file)

if personal_note:
  print(f"Your personal note: {personal_note}")
